# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [2]:
%pip install -r requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.18.0 requires fsspec[http]<=2024.2.0,>=2023.1.0, but you have fsspec 2024.6.1 which is incompatible.
gradio 4.44.1 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.
langchain-experimental 0.3.4 requires langchain-community<0.4.0,>=0.3.0, but you have langchain-community 0.0.29 which is incompatible.
langchain-experimental 0.3.4 requires langchain-core<0.4.0,>=0.3.28, but you have langchain-core 0.1.53 which is incompatible.
scrapegraph-py 1.9.0 requires python-dotenv>=1.0.1, but you have python-dotenv 1.0.0 which is incompatible.
tensorflow-intel 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 2


INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [4]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [5]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [6]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [10]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [11]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [12]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [13]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: Artificial Intelligence

Introduction:
- Brief overview of what artificial intelligence is and its impact on society
- Mention of the latest trends in AI technology
- Introduction of key players in the AI industry
- Mention of recent noteworthy news related to AI

Key Points:
1. Latest Trends in Artificial Intelligence:
- Advancements in natural language processing
- Growth of AI-powered virtual assistants
- Expansion of AI in healthcare and autonomous vehicles
- Increased focus on ethical AI d

- Display the results of your execution as markdown in the notebook.

In [14]:
from IPython.display import Markdown
Markdown(result)

# The Evolution of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has become an integral part of our society, revolutionizing various industries and shaping the way we interact with technology. From advancements in natural language processing to the growth of AI-powered virtual assistants, the latest trends in AI technology are continuously pushing the boundaries of innovation. Key players in the AI industry, such as Google with its DeepMind project and IBM Watson, are at the forefront of developing cutting-edge AI solutions. Recent noteworthy news related to AI, including breakthroughs in AI-powered drug discovery and the ethical concerns surrounding facial recognition technology, highlight the impact of AI on society.

## Latest Trends in Artificial Intelligence

The field of AI is rapidly evolving, with advancements in natural language processing enabling machines to understand and generate human language more accurately than ever before. The growth of AI-powered virtual assistants, such as Amazon's Alexa technology, has transformed the way we interact with our devices, making everyday tasks more convenient and efficient. AI is also making significant strides in healthcare and autonomous vehicles, with the potential to revolutionize these industries in the near future. As AI continues to advance, there is an increased focus on ethical AI development, ensuring that AI technologies are developed and used responsibly.

## Key Players in the AI Industry

Google, with its DeepMind project, is a major player in the AI industry, pushing the boundaries of AI research and development. IBM Watson is another key player, known for its cognitive computing capabilities and applications in various industries. Amazon's Alexa technology has gained widespread popularity for its AI-powered virtual assistant capabilities, while Tesla is leading the way in self-driving cars and autonomous vehicles. These key players are driving innovation in AI and shaping the future of technology.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in AI-powered drug discovery have the potential to revolutionize the pharmaceutical industry, speeding up the process of developing new drugs and treatments. AI algorithms are also being used to predict natural disasters, helping to mitigate the impact of these catastrophic events. However, ethical concerns surrounding facial recognition technology have raised important questions about privacy and surveillance. AI is also being used for fraud detection in financial institutions, improving security and preventing financial crimes.

In conclusion, Artificial Intelligence is a rapidly evolving field with the potential to transform industries and improve our daily lives. By staying informed about the latest trends in AI, exploring the ethical implications of AI technology, and learning more about key players in the industry, we can better understand the impact of AI on society. I encourage readers to share their thoughts on AI developments and continue to engage with this exciting and dynamic field of technology.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "Matching Volunteer Opportunities with One's Life Situation"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Matching Volunteer Opportunities with One's Life Situation.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: Matching Volunteer Opportunities with One's Life Situation

Introduction:
- Brief overview of the importance of finding volunteer opportunities that align with one's life situation
- Explanation of how matching volunteer opportunities can lead to a more fulfilling experience for both the volunteer and the organization

Key Points:
1. Latest Trends:
- Remote volunteering opportunities due to the rise of virtual work
- Flexible volunteering options to accommodate 

In [16]:
Markdown(result)

# Matching Volunteer Opportunities with One's Life Situation

## Introduction
Volunteering is a noble way to give back to the community and make a positive impact on society. However, finding the right volunteer opportunities that align with one's life situation is crucial for a fulfilling experience for both the volunteer and the organization. In this blog post, we will explore the latest trends, key players in the volunteer sector, noteworthy news, target audience analysis, and provide tips on how to find volunteer opportunities that match your life situation.

## Latest Trends in Matching Volunteer Opportunities
In recent years, there has been a shift towards remote volunteering opportunities due to the rise of virtual work. This allows individuals to volunteer from the comfort of their homes, making it more accessible for those with busy schedules or limited availability. Additionally, flexible volunteering options have become increasingly popular to accommodate various commitments such as work or caregiving responsibilities. Skills-based volunteering is also on the rise, as it allows volunteers to leverage their expertise and maximize their impact on the organizations they support.

## Key Players in the Volunteer Sector
Major volunteer platforms like VolunteerMatch and Idealist play a crucial role in connecting volunteers with opportunities that match their skills and interests. Nonprofit organizations also offer diverse volunteer programs to cater to different causes and initiatives. Furthermore, corporate companies are promoting employee volunteer programs to encourage their staff to give back to the community and make a difference in society.

## Noteworthy News in the Volunteering Community
There are numerous success stories of individuals who have found meaningful volunteer opportunities that match their life situations, leading to personal growth and a sense of fulfillment. Impactful volunteer projects have made a significant difference in local communities, highlighting the importance of volunteerism in driving positive change. Updates on new initiatives aimed at connecting volunteers with suitable opportunities continue to emerge, providing more options for individuals to get involved in meaningful causes.

Stay tuned for the next segments of this blog post, where we will delve deeper into the target audience analysis, how to find volunteer opportunities that match your life situation, the benefits of matching volunteer opportunities, and provide tips for getting started with volunteer matching. Stay engaged and discover how you can make a difference in your community through volunteering.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).